#**Importing required libraries**

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
import time
import copy
from PIL import Image
from torch import nn, optim
from torchvision import transforms

In [3]:
!pip install split-folders
import splitfolders as sf
import torch
import os

#**Downloading inatuaralist dataset zip file from drive**

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id = '19EA0yl7PM8i6aQTdhH0OiEyLcTsf6hmx'

In [6]:
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('nature_12K.zip')

#**Unzipping the content and distributing in train, validation and test folders**

In [7]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
!unzip 'nature_12K.zip'

In [9]:
road='/content/inaturalist_12K/'
roadtrn=road+"train"
op= road+"trainvalsplit"
sf.fixed(roadtrn, op, seed=1337, fixed=100, oversample=False, group_prefix=None)

Copying files: 9999 files [01:20, 124.58 files/s]


#**Preprocessing the images**

In [4]:
#preprocessing train data
train_data = []
train_label = []
path = '/content/inaturalist_12K/trainvalsplit/train/'
items = os.listdir(path)
items.sort()

data_augmentation = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


for i in range(10):
    image_folder_path = path + items[i]
    #print(image_folder_path)
    image_names = os.listdir(image_folder_path)
    for each_image in image_names:
        if each_image.endswith(".jpg"):
            full_path = image_folder_path + '/' + each_image
            image = Image.open(full_path)
            image = image.resize((224,224))
            if image.mode == 'L':
                continue
            normalized_image = data_augmentation(image)
            train_data.append((normalized_image, i))

In [5]:
#preprocessing validation data
val_data = []
val_label = []
path = '/content/inaturalist_12K/trainvalsplit/val/'
items = os.listdir(path)
items.sort()

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

for i in range(10):
    image_folder_path = path + items[i]
    #print(image_folder_path)
    image_names = os.listdir(image_folder_path)
    for each_image in image_names:
        if each_image.endswith(".jpg"):
            full_path = image_folder_path + '/' + each_image
            image = Image.open(full_path)
            image = image.resize((224,224))
            if image.mode == 'L':
                continue
            normalized_image = transform(image)
            val_data.append((normalized_image, i))

In [6]:
# #preprocessing test data
# test_data = []
# test_label = []
# path = '/content/inaturalist_12K/val/'
# items = os.listdir(path)
# items.sort()

# transform = transforms.Compose(
#     [transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# for i in range(10):
#     image_folder_path = path + items[i]
#     #print(image_folder_path)
#     image_names = os.listdir(image_folder_path)
#     for each_image in image_names:
#         if each_image.endswith(".jpg"):
#             full_path = image_folder_path + '/' + each_image
#             image = Image.open(full_path)
#             image = image.resize((224,224))
#             if image.mode == 'L':
#                 continue
#             normalized_image = transform(image)
#             test_data.append((normalized_image, i))

In [7]:
classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 
           'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#**Constructing Convolution Neural Network**

In [9]:
class ConvNet(nn.Module):
    def __init__(self, dropout, channels, activation, ks, fc_neurons):
        super(ConvNet, self).__init__()

        self.channels = channels
        if activation == 'ReLU':
          self.activation = nn.ReLU()
        elif activation == 'LeakyReLU':
          self.activation = nn.LeakyReLU()
        elif activation == 'ELU':
          self.activation = nn.ELU()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = channels[0], kernel_size = ks[0])
        torch.nn.init.xavier_normal_(self.conv1.weight)
        self.pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        self.conv2 = nn.Conv2d(in_channels = channels[0], out_channels = channels[1], kernel_size = ks[1])
        torch.nn.init.xavier_normal_(self.conv2.weight)
        self.pool2 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        self.conv3 = nn.Conv2d(in_channels = channels[1], out_channels = channels[2], kernel_size = ks[2])
        torch.nn.init.xavier_normal_(self.conv3.weight)
        self.pool3 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        self.conv4 = nn.Conv2d(in_channels = channels[2], out_channels = channels[3], kernel_size = ks[3])
        torch.nn.init.xavier_normal_(self.conv4.weight)
        self.pool4 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        self.conv5 = nn.Conv2d(in_channels = channels[3], out_channels = channels[4], kernel_size = ks[4])
        torch.nn.init.xavier_normal_(self.conv5.weight)
        self.pool5 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(in_features = channels[4]*2*2, out_features = fc_neurons).to(device)
        torch.nn.init.xavier_normal_(self.fc.weight).to(device)        
        self.op = nn.Linear(in_features = fc_neurons, out_features = 10).to(device)

    def forward(self, x):
        x = self.pool1(self.activation(self.conv1(x)))
        x = self.pool2(self.activation(self.conv2(x)))
        x = self.pool3(self.activation(self.conv3(x)))
        x = self.pool4(self.activation(self.conv4(x)))
        x = self.pool5(self.activation(self.conv5(x)))
        
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = self.dropout(x)        
        x = self.activation(self.fc(x))
        x = self.op(x)
        
        return x

In [10]:
#training fucntion for CNN
def train_model(model, criteria, optimizer, num_epochs=5, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs ))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode                
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            if phase == 'train':
                f = train_loader                
            else:
                f = val_loader
            for inputs, labels in f:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = 0
            epoch_acc = 0
            if phase == 'train':
                epoch_loss = running_loss / len(train_data)
                epoch_acc = running_corrects.double() /len(train_data)
            else:
                epoch_loss = running_loss / len(val_data)
                epoch_acc = running_corrects.double() / len(val_data)
            epoch_acc*=100
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best validation acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    # return model

#**Defining data loaders and training the model**

In [11]:
torch.manual_seed(50)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)

# test_loader = torch.utils.data.DataLoader(test_data, batch_size=200, shuffle=True)

val_loader = torch.utils.data.DataLoader(val_data, batch_size=100, shuffle=False)

In [13]:
model = ConvNet(0.25, [32, 64, 128, 256, 512], 'ReLU', [5]*5, 512).to(device)
num_epochs = 10 
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003, betas=(0.9, 0.999))
train_model(model, criteria, optimizer, num_epochs, 'cuda')

Epoch 1/10
----------
train Loss: 2.2321 Acc: 16.1609
valid Loss: 2.1616 Acc: 18.7187
Epoch 2/10
----------
train Loss: 2.1268 Acc: 22.0518
valid Loss: 2.0661 Acc: 26.4264
Epoch 3/10
----------
train Loss: 2.0277 Acc: 27.1646
valid Loss: 1.9995 Acc: 27.5275
Epoch 4/10
----------
train Loss: 1.9536 Acc: 30.5546
valid Loss: 1.9475 Acc: 29.4294
Epoch 5/10
----------
train Loss: 1.8836 Acc: 33.6112
valid Loss: 1.8695 Acc: 31.7317
Epoch 6/10
----------
train Loss: 1.8257 Acc: 35.7008
valid Loss: 1.8540 Acc: 33.5335
Epoch 7/10
----------
train Loss: 1.7763 Acc: 37.4347
valid Loss: 1.8058 Acc: 34.9349
Epoch 8/10
----------
train Loss: 1.7131 Acc: 40.3579
valid Loss: 1.7769 Acc: 36.2362
Epoch 9/10
----------
train Loss: 1.6535 Acc: 42.6253
valid Loss: 1.8003 Acc: 37.5375
Epoch 10/10
----------
train Loss: 1.5842 Acc: 44.7260
valid Loss: 1.7661 Acc: 37.4374
Training complete in 4m 15s
Best val Acc: 37.537538
